<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250628%5D%20Hybrid%20MLP-LSTM%20with%20FGSM%20%26%20PGD%20on%20CIC%20IoMT%202024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hybrid MLP-LSTM with FGSM & PGD on CIC IoMT 2024 Dataset**

In [ ]:
import pandas as pd

# Load the CSVs
train_df = pd.read_csv('CIC_IoMT_2024_WiFi_MQTT_train.csv')
test_df = pd.read_csv('CIC_IoMT_2024_WiFi_MQTT_test.csv')

# Combine them
full_df = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
full_df['Label_Binary'] = full_df['label'].apply(lambda x: 0 if 'Benign' in x else 1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
full_df['Label_Multiclass'] = le.fit_transform(full_df['label'])

In [ ]:
!pip uninstall -y foolbox
!pip uninstall -y foolbox-native
!pip install foolbox==3.3.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.6 MB/s eta 0:00:00


In [ ]:
# Step 1: Data Preprocessing for CIC IoMT 2024 Dataset
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
# Load and combine the CIC IoMT 2024 datasets
print("🔹 Loading CIC IoMT 2024 WiFi MQTT Dataset...")
train_df = pd.read_csv('CIC_IoMT_2024_WiFi_MQTT_train.csv')
test_df = pd.read_csv('CIC_IoMT_2024_WiFi_MQTT_test.csv')

🔹 Loading CIC IoMT 2024 WiFi MQTT Dataset...


In [ ]:
# Combine datasets
full_df = pd.concat([train_df, test_df], ignore_index=True)
print(f"Combined dataset shape: {full_df.shape}")

Combined dataset shape: (8775013, 46)


In [ ]:
# Handle missing values
full_df = full_df.dropna()
print(f"Dataset shape after removing NaN: {full_df.shape}")

Dataset shape after removing NaN: (8775013, 46)


In [ ]:
# Label encoding for both binary and multiclass
# Binary Classification (0: Benign, 1: Attack)
full_df['Label_Binary'] = full_df['label'].apply(lambda x: 0 if 'Benign' in x else 1)

# Multiclass Classification
le = LabelEncoder()
full_df['Label_Multiclass'] = le.fit_transform(full_df['label'])

In [ ]:
# Display class distributions
print("\nOriginal Class Distribution:")
print(full_df['label'].value_counts())
print("\nBinary Class Distribution:")
print(full_df['Label_Binary'].value_counts())
print("\nMulticlass Class Distribution:")
print(full_df['Label_Multiclass'].value_counts())


Original Class Distribution:
label
TCP_IP-DDoS-UDP2_train      207295
TCP_IP-DDoS-UDP3_train      206604
TCP_IP-DDoS-UDP4_train      206343
TCP_IP-DDoS-UDP1_train      206170
TCP_IP-DDoS-UDP1_test       205654
                             ...  
MQTT-Malformed_Data_test      1747
ARP_Spoofing_test             1744
Recon-VulScan_test            1034
Recon-Ping_Sweep_train         740
Recon-Ping_Sweep_test          186
Name: count, Length: 72, dtype: int64

Binary Class Distribution:
Label_Binary
1    8544674
0     230339
Name: count, dtype: int64

Multiclass Class Distribution:
Label_Multiclass
45    207295
46    206604
47    206343
43    206170
42    205654
       ...  
12      1747
0       1744
20      1034
17       740
16       186
Name: count, Length: 72, dtype: int64


In [ ]:
# Choose classification approach (Binary or Multiclass)
# For manuscript revision, start with Binary classification as recommended
CLASSIFICATION_TYPE = 'binary'  # Change to 'multiclass' if needed

if CLASSIFICATION_TYPE == 'binary':
    target_column = 'Label_Binary'
    print("🔹 Using Binary Classification (Normal vs Attack)")
else:
    target_column = 'Label_Multiclass'
    print("🔹 Using Multiclass Classification")

# Prepare features and target
feature_columns = [col for col in full_df.columns if col not in ['Class', 'Label_Binary', 'Label_Multiclass']]
X = full_df[feature_columns]
y = full_df[target_column]

# Handle categorical variables in features
print("🔹 Processing categorical variables...")
categorical_columns = X.select_dtypes(include=['object']).columns
for column in categorical_columns:
    le_temp = LabelEncoder()
    X[column] = le_temp.fit_transform(X[column].astype(str))

# Convert to numpy arrays
X = X.values
y = y.values

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Original target distribution: {np.bincount(y)}")

🔹 Using Binary Classification (Normal vs Attack)
🔹 Processing categorical variables...


/tmp/ipython-input-13-3040351682.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le_temp.fit_transform(X[column].astype(str))


Feature matrix shape: (8775013, 46)
Target vector shape: (8775013,)
Original target distribution: [ 230339 8544674]


In [ ]:
# Standardize features
print("🔹 Standardizing features...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Stratified train-test split to ensure both classes in training and testing
print("🔹 Performing stratified train-test split...")
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")
print(f"Training target distribution: {np.bincount(y_train)}")
print(f"Testing target distribution: {np.bincount(y_test)}")

🔹 Standardizing features...
🔹 Performing stratified train-test split...
Training set shape: (7020010, 46)
Testing set shape: (1755003, 46)
Training target distribution: [ 184271 6835739]
Testing target distribution: [  46068 1708935]


In [ ]:
# Apply SMOTE for class balance (only on training data)
print("🔹 Applying SMOTE for class balance...")
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"Balanced training set shape: {X_train_balanced.shape}")
print(f"Balanced training target distribution: {np.bincount(y_train_balanced)}")

🔹 Applying SMOTE for class balance...
Balanced training set shape: (13671478, 46)
Balanced training target distribution: [6835739 6835739]


In [ ]:
# Save preprocessed data
preprocessed_data = {
    'X_train': X_train_balanced,
    'y_train': y_train_balanced,
    'X_test': X_test,
    'y_test': y_test,
    'scaler': scaler,
    'label_encoder': le,
    'feature_columns': feature_columns
}

with open("preprocessed_data_ciciomt.pkl", "wb") as f:
    pickle.dump(preprocessed_data, f)

print("✅ Preprocessed CIC IoMT 2024 data saved!")

# Step 2: Logistic Regression Feature Extraction
print("\n🔹 Step 2: Logistic Regression Feature Extraction")

✅ Preprocessed CIC IoMT 2024 data saved!

🔹 Step 2: Logistic Regression Feature Extraction


In [ ]:
# Load preprocessed data
with open("preprocessed_data_ciciomt.pkl", "rb") as f:
    data = pickle.load(f)

X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']

# Train Logistic Regression for Feature Extraction
print("🔹 Training Logistic Regression for feature extraction...")
if CLASSIFICATION_TYPE == 'binary':
    lr_model = LogisticRegression(
        random_state=42,
        max_iter=1000,
        solver='liblinear'  # Better for binary classification
    )
else:
    lr_model = LogisticRegression(
        multi_class='multinomial',
        solver='lbfgs',
        max_iter=1000,
        random_state=42
    )

lr_model.fit(X_train, y_train)

# Extract probabilities for BiLSTM
train_lr_features = lr_model.predict_proba(X_train)
test_lr_features = lr_model.predict_proba(X_test)

# Reshape for LSTM input (add time dimension)
train_lr_features = np.expand_dims(train_lr_features, axis=1)
test_lr_features = np.expand_dims(test_lr_features, axis=1)

# One-hot encode labels for neural network
if CLASSIFICATION_TYPE == 'binary':
    num_classes = 2
else:
    num_classes = len(np.unique(np.concatenate([y_train, y_test])))

y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_test_encoded = to_categorical(y_test, num_classes=num_classes)

print(f"LR feature shape: {train_lr_features.shape}")
print(f"Number of classes: {num_classes}")

# Save features
feature_data = {
    'train_lr_features': train_lr_features,
    'test_lr_features': test_lr_features,
    'y_train_encoded': y_train_encoded,
    'y_test_encoded': y_test_encoded,
    'lr_model': lr_model,
    'num_classes': num_classes
}

with open("lr_features_ciciomt.pkl", "wb") as f:
    pickle.dump(feature_data, f)

print("✅ Logistic Regression features saved!")

# Step 3: Hybrid LR-BiLSTM Training
print("\n🔹 Step 3: Hybrid LR-BiLSTM Training")

🔹 Training Logistic Regression for feature extraction...
LR feature shape: (13671478, 1, 2)
Number of classes: 2
✅ Logistic Regression features saved!

🔹 Step 3: Hybrid LR-BiLSTM Training


In [ ]:
# Load Logistic Regression features
with open("lr_features_ciciomt.pkl", "rb") as f:
    feature_data = pickle.load(f)

train_lr_features = feature_data['train_lr_features']
test_lr_features = feature_data['test_lr_features']
y_train_encoded = feature_data['y_train_encoded']
y_test_encoded = feature_data['y_test_encoded']
num_classes = feature_data['num_classes']

print(f"Input shape for BiLSTM: {train_lr_features.shape}")
print(f"Output classes: {num_classes}")

# Define BiLSTM Model
print("🔹 Building BiLSTM model...")
bilstm_model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.0001)),
                  input_shape=(1, train_lr_features.shape[2])),
    Dropout(0.2),
    Bidirectional(LSTM(32, kernel_regularizer=l2(0.0001))),
    Dropout(0.2),
    Dense(num_classes, activation="softmax")
])

# Compile model
bilstm_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Model summary
print("🔹 Model Architecture:")
bilstm_model.summary()

# Train model
print("🔹 Training BiLSTM model...")
history = bilstm_model.fit(
    train_lr_features, y_train_encoded,
    validation_split=0.2,
    epochs=5,
    batch_size=64,
    verbose=1
)

# Evaluate on test set
print("🔹 Evaluating on test set...")
test_loss, test_accuracy = bilstm_model.evaluate(test_lr_features, y_test_encoded, verbose=0)
print(f"Test Accuracy: {test_accuracy:.10f}")
print(f"Test Loss: {test_loss:.10f}")

# Generate predictions for detailed metrics
y_pred_proba = bilstm_model.predict(test_lr_features)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = np.argmax(y_test_encoded, axis=1)

# Calculate detailed metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"\n🔹 Detailed Performance Metrics:")
print(f"Accuracy: {accuracy:.10f}")
print(f"Precision: {precision:.10f}")
print(f"Recall: {recall:.10f}")
print(f"F1-Score: {f1:.10f}")
print(f"Detection Rate: {recall:.10f}")

# Save trained model
bilstm_model.save("bilstm_model_ciciomt.h5")
print("✅ BiLSTM Model saved!")

# Step 4: Visualization
print("\n🔹 Step 4: Training Visualization")

Input shape for BiLSTM: (13671478, 1, 2)
Output classes: 2
🔹 Building BiLSTM model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


🔹 Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 1, 128)         │        34,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,650 (295.51 KB)

 Trainable params: 75,650 (295.51 KB)

 Non-trainable params: 0 (0.00 B)

🔹 Training BiLSTM model...
Epoch 1/50
170894/170894 ━━━━━━━━━━━━━━━━━━━━ 1612s 9ms/step - accuracy: 0.9979 - loss: 0.0136 - val_accuracy: 0.9993 - val_loss: 0.0056
Epoch 2/50
170894/170894 ━━━━━━━━━━━━━━━━━━━━ 1632s 10ms/step - accuracy: 0.9983 - loss: 0.0108 - val_accuracy: 0.9995 - val_loss: 0.0034
Epoch 3/50
170894/170894 ━━━━━━━━━━━━━━━━━━━━ 1625s 10ms/step - accuracy: 0.9983 - loss: 0.0106 - val_accuracy: 0.9993 - val_loss: 0.0069
Epoch 4/50
170894/170894 ━━━━━━━━━━━━━━━━━━━━ 1632s 10ms/step - accuracy: 0.9983 - loss: 0.0106 - val_accuracy: 0.9993 - val_loss: 0.0045
Epoch 5/50
170894/170894 ━━━━━━━━━━━━━━━━━━━━ 1643s 10ms/step - accuracy: 0.9983 - loss: 0.0106 - val_accuracy: 0.9993 - val_loss: 0.0052
Epoch 6/50
170894/170894 ━━━━━━━━━━━━━━━━━━━━ 1648s 10ms/step - accuracy: 0.9983 - loss: 0.0105 - val_accuracy: 0.9992 - val_loss: 0.0076
Epoch 7/50
170894/170894 ━━━━━━━━━━━━━━━━━━━━ 1726s 10ms/step - accuracy: 0.9983 - loss: 0.0107 - val_accuracy: 0.9993 - val_loss: 0.0059
Epoch 8/

In [ ]:
# Plot Training & Validation Accuracy
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label="Training Accuracy", color="blue")
plt.plot(history.history['val_accuracy'], label="Validation Accuracy", color="red", linestyle="dashed")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training & Validation Accuracy Over Epochs")
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label="Training Loss", color="blue")
plt.plot(history.history['val_loss'], label="Validation Loss", color="red", linestyle="dashed")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training & Validation Loss Over Epochs")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Classification Report
print("\n🔹 Classification Report:")
if CLASSIFICATION_TYPE == 'binary':
    target_names = ['Benign', 'Attack']
else:
    target_names = [f'Class_{i}' for i in range(num_classes)]

print(classification_report(y_true, y_pred, target_names=target_names))

# Confusion Matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true, y_pred)
import seaborn as sns
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Clean Test Data')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

print("✅ Basic model training and evaluation completed!")

# Step 5: Adversarial Training with FGSM and PGD
print("\n🔹 Step 5: Adversarial Attack Evaluation")

In [ ]:
import foolbox as fb

# Load the trained model
bilstm_model = tf.keras.models.load_model("bilstm_model_ciciomt.h5")

# Convert model to Foolbox format
fmodel = fb.TensorFlowModel(bilstm_model, bounds=(0, 1))

# Define attacks
attack_fgsm = fb.attacks.FGSM()
attack_pgd = fb.attacks.LinfPGD(steps=20)

# Function to generate adversarial examples
def generate_adversarial_examples(attack, X_test, y_test, eps=0.02, batch_size=1024):
    num_samples = X_test.shape[0]
    adv_examples = []

    for i in range(0, num_samples, batch_size):
        batch_X = X_test[i:i + batch_size]
        batch_y = np.argmax(y_test[i:i + batch_size], axis=1)
        batch_X_tf = tf.convert_to_tensor(batch_X, dtype=tf.float32)

        # Generate adversarial examples
        adv_batch, *_ = attack(fmodel, batch_X_tf, batch_y, epsilons=eps)
        adv_examples.append(adv_batch.numpy())

    return np.vstack(adv_examples)

# Generate adversarial examples
print("🔹 Generating FGSM Adversarial Examples...")
adv_fgsm_test = generate_adversarial_examples(attack_fgsm, test_lr_features, y_test_encoded, eps=0.01)

print("🔹 Generating PGD Adversarial Examples...")
adv_pgd_test = generate_adversarial_examples(attack_pgd, test_lr_features, y_test_encoded, eps=0.02)

# Evaluate attacks
def evaluate_attack(adv_examples, attack_name):
    adv_predictions = bilstm_model.predict(adv_examples)
    adv_predictions_labels = np.argmax(adv_predictions, axis=1)
    true_labels = np.argmax(y_test_encoded, axis=1)

    accuracy = accuracy_score(true_labels, adv_predictions_labels)
    precision = precision_score(true_labels, adv_predictions_labels, average='weighted')
    recall = recall_score(true_labels, adv_predictions_labels, average='weighted')
    f1 = f1_score(true_labels, adv_predictions_labels, average='weighted')

    print(f"\n✅ {attack_name} Attack Performance Metrics:")
    print(f"✅ Accuracy: {accuracy:.10f}")
    print(f"✅ Precision: {precision:.10f}")
    print(f"✅ Recall: {recall:.10f}")
    print(f"✅ F1-Score: {f1:.10f}")

    return accuracy, precision, recall, f1

# Evaluate both attacks
fgsm_metrics = evaluate_attack(adv_fgsm_test, "FGSM")
pgd_metrics = evaluate_attack(adv_pgd_test, "PGD")

# Confusion matrices for adversarial attacks
def plot_confusion_matrix_adv(y_true, y_pred, title="Confusion Matrix"):
    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Reds',
                xticklabels=target_names, yticklabels=target_names)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(title)
    plt.show()

print("\n🔹 Confusion Matrix for FGSM Adversarial Examples")
plot_confusion_matrix_adv(np.argmax(y_test_encoded, axis=1),
                         np.argmax(bilstm_model.predict(adv_fgsm_test), axis=1),
                         "FGSM Attack Confusion Matrix")

print("\n🔹 Confusion Matrix for PGD Adversarial Examples")
plot_confusion_matrix_adv(np.argmax(y_test_encoded, axis=1),
                         np.argmax(bilstm_model.predict(adv_pgd_test), axis=1),
                         "PGD Attack Confusion Matrix")

print("✅ Complete CIC IoMT 2024 analysis finished!")

# Step 6: Feature Importance Analysis (Addressing Reviewer Concerns)
print("\n🔹 Step 6: Feature Importance Analysis")

In [ ]:
# Get feature importance from Logistic Regression
lr_model = feature_data['lr_model']
feature_importance = np.abs(lr_model.coef_[0]) if CLASSIFICATION_TYPE == 'binary' else np.mean(np.abs(lr_model.coef_), axis=0)

# Create feature importance DataFrame
feature_names = data['feature_columns']
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

# Plot top 20 most important features
plt.figure(figsize=(12, 8))
top_features = importance_df.head(20)
plt.barh(range(len(top_features)), top_features['Importance'])
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Feature Importance (Absolute Coefficient Value)')
plt.title('Top 20 Most Important Features for IoMT Intrusion Detection')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("Top 10 Most Important Features:")
print(importance_df.head(10))

print("\n✅ All analysis completed successfully!")